Challenge extracted from https://medium.com/art-of-data-engineering/can-you-crack-this-sql-query-in-30-minutes-2809c054bc52

<span style="color:blue;">

## Problem Statement
A company has secured a $1 million budget to build a data team and develop its data platform. The hiring team is working with a dataset called “Candidate,” which contains a comprehensive list of potential hires, categorized as senior and junior data engineers, along with details about their salary expectations. All senior and junior candidates have similar skills within their respective levels and are considered eligible for hiring.

The goal of the hiring team is -

1. To hire the maximum number of seniors first.

2. After hiring the maximum number of seniors, the remaining budget should be used to hire as many juniors as possible.

<b>You need to write a SQL query to get the optimal list of all the candidates who can join the data team based on the above goals?</b>
</span>

In [12]:
import sqlite3
conn = sqlite3.connect(':memory:')
sql = conn.cursor()

In [61]:
# Table Design and Sample Data
sql.execute('''
create table if not exists 
candidate (candidate_id int, experience String(50), salary int)
''')
sql.execute("DELETE FROM candidate")

sql.execute("""
INSERT INTO candidate (candidate_id, experience, salary) VALUES
(1, 'Senior', 200000),
(2, 'Senior', 95000),
(3, 'Senior', 110000),
(4, 'Senior', 105000),
(5, 'Senior', 120000),
(6, 'Senior', 185000),
(7, 'Senior', 190000),
(8, 'Senior', 115000),
(9, 'Senior', 180000),
(10, 'Senior', 98000),
(11, 'Junior', 70000),
(12, 'Junior', 75000),
(13, 'Junior', 60000),
(14, 'Junior', 61000),
(15, 'Junior', 55000)
""")

conn.commit()  #Remember to commit!!! :-)


In [63]:
# Check the data just loaded
import pandas as pd
pd.read_sql_query("SELECT * FROM candidate ORDER BY candidate_id", conn)

,candidate_id,experience,salary
0,1,Senior,200000
1,2,Senior,95000
2,3,Senior,110000
3,4,Senior,105000
4,5,Senior,120000
5,6,Senior,185000
6,7,Senior,190000
7,8,Senior,115000
8,9,Senior,180000
9,10,Senior,98000


In [71]:
# The solution of the problem:

challenge_2_solution = pd.read_sql_query("""
WITH
--
--> $1 million budget to build a data team
--
TotalBudget AS (
SELECT 1000000 AS Budget    
)
--
--> Rolling sum of salary expectations by senior and junior experience starting from the candidate asking less money
--> <CumulativeSalary> means the budget allocated up to the i-th candidate for that skill
--
,CumulativeSalaryByCandidate AS ( 
SELECT 
  candidate_id
  ,experience
  ,ROW_NUMBER() OVER (PARTITION BY experience ORDER BY salary, candidate_id  ASC ) AS CandidateSequence 
  ,salary
  ,SUM(salary) OVER (PARTITION BY experience ORDER BY salary, candidate_id  ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS CumulativeSalary
FROM candidate
)
--
-- The budget allocated for junior is the money not used to hire the senior candidates
-- The <hiring seniors> will be the candidates who have the <CumulativeSalary> less or equal the initial <TotalBudget>
-- The biggest <CumulativeSalary> of the <hiring seniors> is the maximum budget "assignable" for seniors
--
-- ==> Remaining budget for the junior is <Total budget> - <CumulativeSalary> of the "last" and biggest <hiring senior> row
-- 
, BudgetForExperience AS (
SELECT 
  'Junior'                                    AS experience 
  ,MIN(TotalBudget.Budget - CumulativeSalary) AS Budget
FROM 
  CumulativeSalaryByCandidate 
  ,TotalBudget
WHERE experience = "Senior"
AND CumulativeSalary <= TotalBudget.Budget --> <hiring seniors>
UNION ALL
SELECT
  'Senior'                                    AS experience 
  ,Budget                                     AS Budget
FROM 
  TotalBudget
)
select 
  CumulativeSalaryByCandidate.experience
  ,CumulativeSalaryByCandidate.CandidateSequence
  ,CumulativeSalaryByCandidate.candidate_id
  ,CumulativeSalaryByCandidate.Salary
  ,SUM(CumulativeSalaryByCandidate.Salary) OVER (ORDER BY CumulativeSalaryByCandidate.experience DESC, CumulativeSalaryByCandidate.Salary) AS UsedBudget 
FROM CumulativeSalaryByCandidate
INNER JOIN BudgetForExperience ON (CumulativeSalaryByCandidate.experience = BudgetForExperience.experience)
WHERE CumulativeSalaryByCandidate.CumulativeSalary <= BudgetForExperience.Budget
ORDER BY CumulativeSalaryByCandidate.experience DESC, CumulativeSalaryByCandidate.Salary

/*
The Query has been designed to facilitate the editing of a new budget value: if you have more budget you need only to change the value of TotalBudget table at the head of the CTE
*/
""", conn)

challenge_2_solution

,experience,CandidateSequence,candidate_id,salary,UsedBudget
0,Senior,1,2,95000,95000
1,Senior,2,10,98000,193000
2,Senior,3,4,105000,298000
3,Senior,4,3,110000,408000
4,Senior,5,8,115000,523000
5,Senior,6,5,120000,643000
6,Senior,7,9,180000,823000
7,Junior,1,15,55000,878000
8,Junior,2,13,60000,938000
9,Junior,3,14,61000,999000
